### Install Prerequisite

In [1]:
!pip install -q texthero seaborn scikit-multilearn transformers tensorflow-text tf-models-official
!pip install gradio jinja2

### Importing Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('C:/Users/JOY/Desktop/Final Project')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from keras import backend as K
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 

import gradio as gr
import Caribe as cb

J:\Program Files\Anaconda3\envs\python37\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,
J:\Program Files\Anaconda3\envs\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

### Data Loading

In [3]:
df = pd.read_csv(os.path.join('datasets/jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

column_names=["short","long"]
df1 = pd.read_csv("datasets/abbrevations.csv",names=column_names)
df2 = pd.read_csv("datasets/emoji_df.csv")
df3 = pd.read_csv("datasets/twitterSlang.csv")

### Data Exploration

In [4]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

df.head(5)

Number of rows: 159571
Number of columns: 8


,id,comment_text,Toxic,Severe Toxic,Obscene,Threat,Insult,Identity Hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


### Train Test Split

In [5]:
train, test = train_test_split(df, test_size = 0.3, random_state = 1)

In [6]:
print("Train:", train.shape)
print("Test:", test.shape)

Train: (111699, 8)
Test: (47872, 8)


### Vectorization

In [7]:
labels = ['Toxic','Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate']

x_train = train.loc[:,'comment_text']
y_train = train[labels]

x_test = test.loc[:,'comment_text']
y_test = test[labels]

In [8]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=1000)
tfidf_vec.fit(x_train)

TfidfVectorizer(max_features=1000, min_df=2, ngram_range=(1, 2))

In [9]:
# trasforming train and test
train_tfidf = tfidf_vec.transform(x_train)
test_tfidf = tfidf_vec.transform(x_test)

### BERT Transfer Learning

In [10]:
train_bert = train['comment_text']
test_bert = test['comment_text']

labels = ['Toxic','Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate']
y_train = train[labels]
y_test = test[labels]

In [20]:
# Preparaing tensorflow dataset to feed into the model
batch_size = 32
seed = 42

train_ds = tf.data.Dataset.from_tensor_slices((train_bert.values, y_train.values)).shuffle(50000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_bert.values, y_test.values)).shuffle(50000).batch(batch_size)

### Loading the models form tensorflow hub

In [21]:
tfhub_handle_encoder = 'https://tfhub.dev/jeongukjae/distilbert_en_uncased_L-6_H-768_A-12/1'
tfhub_handle_preprocess = 'https://tfhub.dev/jeongukjae/distilbert_en_uncased_preprocess/2'

### Creating Model

In [22]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(500, activation='relu')(net)
  net = tf.keras.layers.Dense(6, activation= "sigmoid" , name='classifier')(net)
  return tf.keras.Model(text_input, net)

### Precision Recall F1

In [40]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Summarize Model

In [24]:
classifier_model = build_classifier_model()
classifier_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                          
                                                                                                  
 BERT_encoder (KerasLayer)      {'encoder_outputs':  66362880    ['preprocessing[0][0]',    

In [25]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.metrics.CategoricalAccuracy(), tf.metrics.AUC(multi_label=True), f1_m,precision_m, recall_m]

epochs = 1
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
init_lr = 3e-5

optimizer = optimization.create_optimizer(init_lr=init_lr,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')

### Compile the Model

In [26]:
classifier_model.compile(optimizer=optimizer,loss=loss, metrics=metrics)

callbacks = [tf.keras.callbacks.ModelCheckpoint('best_distilbert_model', save_best_only=True),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

### Train Model

In [27]:
bert_history = classifier_model.fit(train_ds, validation_data=test_ds, epochs = epochs, callbacks = callbacks)

6982/6982 [==============================] - ETA: 0s - loss: 0.0512 - categorical_accuracy: 0.8890 - auc_1: 0.9684 - f1_m: 0.7400 - precision_m: 0.8171 - recall_m: 0.7317

INFO:tensorflow:Assets written to: best_distilbert_model\assets


INFO:tensorflow:Assets written to: best_distilbert_model\assets


6982/6982 [==============================] - 1916s 274ms/step - loss: 0.0512 - categorical_accuracy: 0.8890 - auc_1: 0.9684 - f1_m: 0.7400 - precision_m: 0.8171 - recall_m: 0.7317 - val_loss: 0.0395 - val_categorical_accuracy: 0.9444 - val_auc_1: 0.9806 - val_f1_m: 0.8437 - val_precision_m: 0.8663 - val_recall_m: 0.8788


### Load the Best Model

In [37]:
bert_model = tf.keras.models.load_model('best_distilbert_model', compile = False)

### Save the Model

In [38]:
# compile the model
bert_model.compile(optimizer=optimizer, loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=metrics)

# Save Model
bert_model.save('models/distilberttoxic1.h5')

### Model Evaluation

In [41]:
loss, accuracy,auc,f1_m,precision_m, recall_m = bert_model.evaluate(test_ds)

2992/2992 [==============================] - 198s 66ms/step - loss: 0.0395 - categorical_accuracy: 0.9444 - auc_1: 0.9806 - f1_m: 0.8273 - precision_m: 0.8466 - recall_m: 0.8656s - loss: 0.0394 - categorical_accuracy: 0.9440 - auc_1: 0.9803 - f1_m: 0.8270 - precisio - ETA: 2s - loss: 0.0395 - categorical_accuracy: 0.9442 - auc_1: 0.9803 - f1_m: 0.8267 - precision_m: 0.8 - ETA: 1s - loss: 0.0395 - categorical_accuracy: 0.9442 - auc_1: 


In [42]:
print(f"Accuracy: {np.round(accuracy,4)}")
print(f"AUC: {np.round(auc,4)}")
print(f"Logloss: {np.round(loss,4)}")
print(f"f1: {np.round(f1_m,4)}")
print(f"precision: {np.round(precision_m,4)}")
print(f"recall: {np.round(recall_m,4)}")

Accuracy: 0.9444
AUC: 0.9806
Logloss: 0.0395
f1: 0.8273
precision: 0.8466
recall: 0.8656


### User Interface

In [32]:
#Importing detoxification function from detox file
from detox import detoxification

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\JOY\AppData\Roaming\Python\Python37\site-packages\transformers\tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
Decoding outputs: 100%|██████████| 1/1 [00:06<00:00,

["I'm tired of the school shootings by black people."]


Decoding outputs: 100%|██████████| 1/1 [00:05<00:00,  5.31s/it]

BLEU score -> 1.2183324802375697e-231 text -> ['I', 'will', 'kill', 'you'] pred -> ['I', 'will', 'not', 'honour', 'you']
CS score -> 0.0 text -> I will kill you pred -> I will not honour you



J:\Program Files\Anaconda3\envs\python37\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
J:\Program Files\Anaconda3\envs\python37\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
J:\Program Files\Anaconda3\envs\python37\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower

### Pre-Processing Filters

In [33]:
def preprocessingfilters(comment):
    print("\nOriginal sentence : " ,comment)

    #Slang 
    for  i in df3.loc[:, 'slang']:
      comment = comment.lower()
      if(i.lower() in comment.split(" ")):
        j = df3[i==df3['slang']]
        k = j.iloc[-1].values

        comment = comment.lower()
        comment = comment.replace(i.lower(), k[1])
        print("Slang Expanded sentence : " ,comment)

    # Abberivation
    for  i in df1.loc[:, 'short']:
      comment = comment.lower()
      if(i.lower() in comment.split(" ")):
        j = df1[i==df1['short']]
        k = j.iloc[-1].values

        comment = comment.lower()
        comment = comment.replace(i.lower(), k[1])
        print("Abberivation Expanded sentence : " ,comment)

    # Emojis
    for  i in df2.loc[:, 'emoji']:
      if(i in comment):
        j = df2[i==df2['emoji']]
        k = j.iloc[-1].values

        comment = comment.replace(i, ", having " +k[1])
        print("Emojis Expanded sentence : " ,comment)

    comment=cb.caribe_corrector(comment)
    print("Correct sentence : " ,comment)
    return comment

### UI Action Function

In [34]:
def score_comment(comment):
    fcomment = preprocessingfilters(comment)
    results = bert_model.predict([comment])
    
    if results[0][0] > 0.1:      
        pred = detoxification(comment)
        pred = (str(pred)[1:-1]).strip('[\]"\',')
        pred = str(pred)
        
    else:
        pred = "Non Toxic Sentence"

    d = []
    for idx, col in enumerate(df.columns[2:]):
        d.append(
            {
                'Catagory': col,
                'Result': results[0][idx] > 0.1,
                'Percent':  round((results[0][idx])*100,2)
            }
        )

    d = pd.DataFrame(d)      
    
    return fcomment,d,pred

### UI Modeling

In [35]:
with gr.Blocks() as interface:
    name = gr.inputs.Textbox(lines=2, placeholder='Enter Your Sentence', label = "Input Sentence")
    greet_btn = gr.Button("Submit")
    output = [gr.Textbox(label="Did you mean?"),gr.Dataframe(label="Toxicity Detection & Classification",
              headers=['Catagory', 'Result', 'Percent']) , gr.Textbox(label="Detoxification")]
    greet_btn.click(fn=score_comment, inputs=name, outputs=output)

J:\Program Files\Anaconda3\envs\python37\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
J:\Program Files\Anaconda3\envs\python37\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
J:\Program Files\Anaconda3\envs\python37\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


### UI Launch

In [36]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7b3769b1e1a5e36f.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
